<a href="https://colab.research.google.com/github/AidanMahoney73/DS-Unit-1-Sprint-2-Statistics/blob/master/module1/LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
### YOUR CODE STARTS HERE

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999 

In [57]:
df_columns = ['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synfuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa']

df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data", 
                 header=None, names=df_columns, na_values='?')

df = df.replace({'y':1, 'n':0})
df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [0]:
#number of voters
rep = df[df['party']=='republican']
dem = df[df['party']=='democrat']

In [68]:
rep.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [69]:
dem.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [51]:
#Loop Quick find for all answers

#example of t test line
#ttest_ind(rep['water-project'], dem['water-project'], nan_policy='omit')

for col in df_columns:
  if col == 'party':
    print("Skipped")
  else:
    print(str(col))
    print(ttest_ind(rep[col], dem[col], nan_policy='omit'))

Skipped
handicapped-infants
Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18)
water-project
Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)
budget
Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)
physician-fee-freeze
Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)
el-salvador-aid
Ttest_indResult(statistic=21.13669261173219, pvalue=5.600520111729011e-68)
religious-groups
Ttest_indResult(statistic=9.737575825219457, pvalue=2.3936722520597287e-20)
anti-satellite-ban
Ttest_indResult(statistic=-12.526187929077842, pvalue=8.521033017443867e-31)
aid-to-contras
Ttest_indResult(statistic=-18.052093200819733, pvalue=2.82471841372357e-54)
mx-missile
Ttest_indResult(statistic=-16.437503268542994, pvalue=5.03079265310811e-47)
immigration
Ttest_indResult(statistic=1.7359117329695164, pvalue=0.08330248490425066)
synfuels
Ttest_indResult(statistic=-8.293603989407588, pvalue=1.5759322301054064e-1

In [53]:
#using fast loop ot find p value is < .01
for col in df_columns:
  if col == 'party':
    print("Skipped")
  else:
    if ttest_ind(rep[col], dem[col], nan_policy='omit')[1] > .01:
      print("Skipped")
    else:
      print(str(col))
      print(ttest_ind(rep[col], dem[col], nan_policy='omit'))

Skipped
handicapped-infants
Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18)
Skipped
budget
Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)
physician-fee-freeze
Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177)
el-salvador-aid
Ttest_indResult(statistic=21.13669261173219, pvalue=5.600520111729011e-68)
religious-groups
Ttest_indResult(statistic=9.737575825219457, pvalue=2.3936722520597287e-20)
anti-satellite-ban
Ttest_indResult(statistic=-12.526187929077842, pvalue=8.521033017443867e-31)
aid-to-contras
Ttest_indResult(statistic=-18.052093200819733, pvalue=2.82471841372357e-54)
mx-missile
Ttest_indResult(statistic=-16.437503268542994, pvalue=5.03079265310811e-47)
Skipped
synfuels
Ttest_indResult(statistic=-8.293603989407588, pvalue=1.5759322301054064e-15)
education
Ttest_indResult(statistic=20.500685724563073, pvalue=1.8834203990450192e-64)
right-to-sue
Ttest_indResult(statistic=13.51064251060933, pvalue=1.2

In [0]:
#Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
# From the for loop we see budget has a p value of less than .01

In [0]:
#Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
# from the for loop we see physician-fee-freeze has a p value less than .01

In [70]:
for col in df_columns:
  if col == 'party':
    print("Skipped")
  else:
    if ttest_ind(rep[col], dem[col], nan_policy='omit')[1] < .01:
      print("Skipped")
    else:
      print(str(col))
      print(ttest_ind(rep[col], dem[col], nan_policy='omit'))

Skipped
Skipped
water-project
Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped
immigration
Ttest_indResult(statistic=1.7359117329695164, pvalue=0.08330248490425066)
Skipped
Skipped
Skipped
Skipped
Skipped
Skipped


In [0]:
#Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)
# from the for loop we see water-project has a p value of > .1